# Define configruations

In [45]:
# Will ignore everything else, only focus on the columns of interest
#columns_of_interest=['Team', 'Number', 'Position', 'Age', 'Height', 'Weight', 'Salary', 'Some Error']
columns_of_interest=None

# Works reversely as columns_of_interest, ignore those columns. 
# It should not use together with column of interest
columns_to_ignore=['College']

# Dara source url
data_src = "https://media.geeksforgeeks.org/wp-content/uploads/nba.csv"
# data_src="https://data.gov.sg/api/action/datastore_search?resource_id=400a3eb4-8702-4050-9700-988bfea7a20f&limit=5&q=title:jones"
# data_src="~/Desktop/iris.data"

# If provided csv data doesn't contains headers, provide as a list of strings
# headers=['sepal_length',	'sepal_width',	'petal_length',	'petal_width',	'species']  # Default none
headers=None

# Foce certian columns data type to be number
# force_to_number = [{'column':'Height', 'method':'replace_dash'}]

The force_to_number configuration can be a bit tricky. If a data set that contains not properly formatted values, I think we can provide some gereic function help user force the value to be numeric value. 
E.g. in the provided data, the value "Height" is saperated by dash '-' ...

But it doesn't work this way, the height have to be converted to centmeters in order to get the proper numeric valye. This way is wrong. 

# Read data

In [46]:
import pandas as pd
from pandas.api.types import is_numeric_dtype

pd.set_option('expand_frame_repr', False)
pd.set_option('display.max_rows', None)

if (headers is not None):
	data = pd.read_csv(data_src, names=headers)
else:
	data = pd.read_csv(data_src)

In [47]:
# Preview the data
pd.options.display.max_columns = 100 # Set max dispaly columns, if the data has too many columns
data.head(20)

,Name,Team,Number,Position,Age,Height,Weight,College,Salary
0,Avery Bradley,Boston Celtics,0.0,PG,25.0,6-2,180.0,Texas,7730337.0
1,Jae Crowder,Boston Celtics,99.0,SF,25.0,6-6,235.0,Marquette,6796117.0
2,John Holland,Boston Celtics,30.0,SG,27.0,6-5,205.0,Boston University,NaN
3,R.J. Hunter,Boston Celtics,28.0,SG,22.0,6-5,185.0,Georgia State,1148640.0
4,Jonas Jerebko,Boston Celtics,8.0,PF,29.0,6-10,231.0,NaN,5000000.0
5,Amir Johnson,Boston Celtics,90.0,PF,29.0,6-9,240.0,NaN,12000000.0
6,Jordan Mickey,Boston Celtics,55.0,PF,21.0,6-8,235.0,LSU,1170960.0
7,Kelly Olynyk,Boston Celtics,41.0,C,25.0,7-0,238.0,Gonzaga,2165160.0
8,Terry Rozier,Boston Celtics,12.0,PG,22.0,6-2,190.0,Louisville,1824360.0
9,Marcus Smart,Boston Celtics,36.0,PG,22.0,6-4,220.0,Oklahoma State,3431040.0


# Pre-process data

### Force data to be number

In [48]:
# Old methods, no in use
def replace_dash(s):
	if ( isinstance(s, str)):
		n = s.replace('-','.')
		n = float(n)
		return n
	else:
		return s

forceMethods = {
	'replace_dash': replace_dash
}


In [49]:
# Old method to resolve the '-' in height

#if(force_to_number is not None):
	#for force_item in force_to_number:
		#column = force_item['column']
		#method = force_item['method']
		#force_func = forceMethods[method]
		#data[column] = data[column].apply(func=force_func)

In [50]:
def feet_inch_to_cm(s):
	if (isinstance(s, str)):
		feet = s.split('-')[0]	
		inch = s.split('-')[1]
		cm = float(feet) * 30.4800 + float(inch) * 2.54
		return cm
	else:
		return s

In [51]:
# This pre-processor only applied to the basketball data set, if use other datasets, this will throw error. 

data['Height'] = data['Height'].apply(feet_inch_to_cm)

# data.head()
# data.shape

### Focus on the columns of interest

In [52]:
# A container for all the error-messages in the description
missing_columns_of_interests = [] 

# Check if there is error in conlums_of_interest, if user interested
# in a column that doesn't exist, record that to generate error message
if(columns_of_interest is not None):
	for column in columns_of_interest:
		if not column in data.columns:
			missing_columns_of_interests.append(column)
	
	# Loop through all columns in data frame, drop those columns that is
	# not interested
	for column in data.columns:
		if not column in columns_of_interest:
			data.drop(column, inplace=True, axis=1)



missing_columns_of_interests

[]

In [53]:
# Ignore the columns to ignore
if (columns_to_ignore is not None):
	for column in columns_to_ignore:
		if column in data.columns:
			data.drop(column, inplace=True, axis=1)

# Overall data description

In [54]:
# Count rows and columns
data_shape = data.shape
total_rows_num = data_shape[0]
total_columns_num = data_shape[1]

print(total_rows_num, total_columns_num)

458 8


In [55]:
# Count total rows that contais a missed value
incomplete_rows = data[data.isna().any(axis=1)]
incomplete_rows_num = incomplete_rows.shape[0]

In [56]:
# Count percentage of incomplete rows
incomplete_rows_pct = incomplete_rows_num/total_rows_num
incomplete_rows_pct

0.026200873362445413

### Calculate missing data for each column
Only calculate the colums set in the columns_of_interest

In [57]:
missing_data = data.isna()
missing_data.head()

,Name,Team,Number,Position,Age,Height,Weight,Salary
0,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False


In [58]:
missing_data_num = missing_data.sum()
missing_data_sum = missing_data_num/len(data)

In [59]:
desc_idx = ['Missing Data (%)','Data Type' ,'Mean', 'Median', 'Min', 'Max', 'Standard Deviation', '25%', '75%', 'Most Appeared', 'Most Appeared Count', 'Unique Values Count']
desc = pd.DataFrame(index=desc_idx)

for k, v in missing_data_sum.items():
	desc.at['Missing Data (%)', k] = round(v,4) * 100

desc

,Name,Team,Number,Position,Age,Height,Weight,Salary
Missing Data (%),0.22,0.22,0.22,0.22,0.22,0.22,0.22,2.62
Data Type,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Median,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Standard Deviation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Most Appeared,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Columte the mean and median of each column and other info

In [60]:
# Data type of each column
for k, v in data.dtypes.items():
	desc.at['Data Type', k] = v

# Mean value
for k, v in data.mean(skipna=True, numeric_only=True).items():
	desc.at['Mean', k] = round(v, 4)

# Median
for k, v in data.median(skipna=True, numeric_only=True).items():
	desc.at['Median', k] = round(v, 4)

# Min value
for k, v in data.min(skipna=True, numeric_only=True).items():
	desc.at['Min', k] = round(v, 4)

# Max value
for k, v in data.max(skipna=True, numeric_only=True).items():
	desc.at['Max', k] = round(v, 4)



# Standard Deviation
for k, v in data.std(skipna=True, numeric_only=True).items():
	desc.at['Standard Deviation', k] = round(v, 4)

quantiles_25 =  data.quantile(q=0.25, numeric_only=True)
quantiles_75 =  data.quantile(q=0.75, numeric_only=True)
for k in data.columns:
	try:
		desc.at['25%',k] = round(quantiles_25[k], 4)
		desc.at['75%',k] = round(quantiles_75[k], 4)
	except:
		pass

# Most frequent value
data_mode = data.mode()
for k in data_mode:
	v = data_mode[k][0]
	desc.at['Most Appeared', k] = v
	w = data[k].value_counts()[v]
	# Count most frequent value
	desc.at['Most Appeared Count', k] = w

for column in data.columns:
	unique = data[column].unique()
	desc.at['Unique Values Count',column] = len(unique) 
desc

,Name,Team,Number,Position,Age,Height,Weight,Salary
Missing Data (%),0.22,0.22,0.22,0.22,0.22,0.22,0.22,2.62
Data Type,object,object,float64,object,float64,float64,float64,float64
Mean,NaN,NaN,17.6783,NaN,26.9387,201.1435,221.523,4842684.1054
Median,NaN,NaN,13.0,NaN,26.0,203.2,220.0,2839073.0
Min,NaN,NaN,0.0,NaN,19.0,175.26,161.0,30888.0
Max,NaN,NaN,99.0,NaN,40.0,220.98,307.0,25000000.0
Standard Deviation,NaN,NaN,15.9661,NaN,4.404,8.7184,26.3683,5229237.5989
25%,NaN,NaN,5.0,NaN,24.0,195.58,200.0,1044792.25
75%,NaN,NaN,25.0,NaN,30.0,208.28,240.0,6500000.0
Most Appeared,Aaron Brooks,New Orleans Pelicans,5.0,SG,24.0,205.74,220.0,947276.0


In [61]:
# removing null values to avoid errors 
# data.dropna(inplace = True) 

# percentile list
# perc =[.20, .40, .60, .80]

# list of dtypes to include
# include =['object', 'float', 'int']

# calling describe method
# desc = data.describe(percentiles = perc, include = include)

# Render the description

In [62]:
from tabulate import tabulate

print('Incomplete rows(%):'.ljust(30), round(incomplete_rows_pct, 4) * 100)
print('Total rows:'.ljust(30), total_rows_num)
print('Incomplete rows cound'.ljust(30), incomplete_rows_num)
print('Total columns of interest:'.ljust(30), total_columns_num )
print('-'*40)
print('Data description:')
print(tabulate(desc, headers='keys', tablefmt='tsv'))
print('-'*40)
if (data[data.isna().any(axis=1)].shape[0] != 0):	
	print('Incomplete rows')
	print(tabulate(data[data.isnull().any(axis=1)],headers='keys', tablefmt='tsv', maxcolwidths=[999, 999]))

Incomplete rows(%):            2.62
Total rows:                    458
Incomplete rows cound          12
Total columns of interest:     8
----------------------------------------
Data description:
                   	Name        	Team                	Number  	Position  	Age    	Height  	Weight  	Salary
Missing Data (%)   	0.22        	0.22                	0.22    	0.22      	0.22   	0.22    	0.22    	2.62
Data Type          	object      	object              	float64 	object    	float64	float64 	float64 	float64
Mean               	nan         	nan                 	17.6783 	nan       	26.9387	201.1435	221.523 	4842684.1054
Median             	nan         	nan                 	13.0    	nan       	26.0   	203.2   	220.0   	2839073.0
Min                	nan         	nan                 	0.0     	nan       	19.0   	175.26  	161.0   	30888.0
Max                	nan         	nan                 	99.0    	nan       	40.0   	220.98  	307.0   	25000000.0
Standard Deviation 	nan         	nan     